In [55]:
import matplotlib.pyplot as plt
from matplotlib  import rc
import pandas as pd
import geopandas as gpd
# from census import Census
# from us import states
import numpy as np
import os

In [3]:
bank_tract = pd.read_csv('../input_data_clean/bank_tract_clean_WITH_CENSUS.csv')

In [8]:
# Get the Arizona tract shapefile and convert to UTM Zone 17N coordinate system
shape_az = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2020/TRACT/tl_2020_04_tract.zip").to_crs(epsg=32617)
shape_ca = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2020/TRACT/tl_2020_06_tract.zip").to_crs(epsg=32617)
shape_all = pd.concat([shape_az, shape_ca], ignore_index=True)
shape_all['census_tract'] = shape_all['GEOID'].astype(np.int64)

In [36]:
shape_all

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,census_tract
0,04,015,950500,04015950500,9505,Census Tract 9505,G5020,S,1255016476,39469850,+35.3440369,-114.3629894,"POLYGON ((-2605753.221 4497432.366, -2605733.2...",4015950500
1,04,015,954900,04015954900,9549,Census Tract 9549,G5020,S,22392503,0,+35.2440284,-114.0595108,"POLYGON ((-2561412.583 4449714.181, -2561223.1...",4015954900
2,04,015,951800,04015951800,9518,Census Tract 9518,G5020,S,3648437,120729,+35.0963207,-114.6188545,"POLYGON ((-2620096.770 4449251.952, -2620077.4...",4015951800
3,04,015,952400,04015952400,9524,Census Tract 9524,G5020,S,295669288,7885448,+34.6247879,-114.3458669,"POLYGON ((-2622343.321 4399330.830, -2622188.9...",4015952400
4,04,027,012100,04027012100,121,Census Tract 121,G5020,S,6508727795,60851,+33.1077362,-113.8376677,"POLYGON ((-2693911.900 4201040.592, -2692586.4...",4027012100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10889,06,065,041407,06065041407,414.07,Census Tract 414.07,G5020,S,1614719,0,+33.9110608,-117.4726363,"POLYGON ((-2950798.156 4420035.268, -2950792.6...",6065041407
10890,06,065,041408,06065041408,414.08,Census Tract 414.08,G5020,S,1422469,0,+33.9042089,-117.4691610,"POLYGON ((-2951447.927 4418581.979, -2951416.2...",6065041408
10891,06,065,041410,06065041410,414.10,Census Tract 414.10,G5020,S,1953877,0,+33.8800908,-117.5305087,"POLYGON ((-2958590.747 4419126.245, -2958563.3...",6065041410
10892,06,013,338101,06013338101,3381.01,Census Tract 3381.01,G5020,S,1128292,0,+37.9489174,-122.0424911,"POLYGON ((-3161121.961 5093496.120, -3161072.5...",6013338101


In [11]:
# merge the shape file with bank_tract
bank_tract = bank_tract.merge(shape_all, how = 'left', on = ['census_tract'],
                              indicator = True, validate= 'm:1') 

In [15]:
# set up map zooms
cities     = ['phx', 'tcs', 'sf', 'la']
phx_zoom   = {'x': [-2.5e6,-2.35e6],     'y' : [4.1e6,4.25e6],    'city':'phx'}
tcs_zoom   = {'x': [-1.17e7, -1.16e7],   'y': [3.63e6, 3.65e6],   'city':'tuscon'}
sf_zoom    = {'x': [-1.362e7, -1.355e7], 'y': [4.33e6, 4.37e6],   'city':'SF'}
la_zoom    = {'x': [-1.287e7, -1.275e7], 'y': [4.033e6, 4.086e6], 'city':'LA'}
citieszoom = [phx_zoom, tcs_zoom, sf_zoom, la_zoom]

In [14]:
#denial_rate, credit_spread, np.log(# loans), np.log($ loans), avg LTV 
bank_tract ["white_rate"] = (bank_tract ["WhitePop"]) / bank_tract ["Tot.Pop"] * 100
bank_tract ['majority-minority'] = bank_tract ["white_rate"] < 50
bank_tract ['below_p10_income'] = bank_tract ["Med.HousehIncome"] < bank_tract["Med.HousehIncome"].quantile(0.1)
bank_tract ['high_hispanic'] = (bank_tract ['HispanicLatinoPop'] / bank_tract['Tot.Pop']) > 0.5
bank_tract ['high_black'] = (bank_tract ['BlackPop'] / bank_tract['Tot.Pop']) > 0.5
bank_tract ['high_asian'] = (bank_tract ['AsianPop'] / bank_tract['Tot.Pop']) > 0.5
bank_tract ['high_white'] = (bank_tract ['WhitePop'] / bank_tract['Tot.Pop']) > 0.5

bank_tract ['log(numloans)'] = np.log(bank_tract['num_approved_loans'])
bank_tract ['log(dol_loans)'] = np.log(bank_tract['avg_approved_loan_size'])


listofshading = ['denial_rate', 'mean_approved_rate_spread', 'log(numloans)', 'log(dol_loans)', 'mean_LTV']

listofredlining = ['majority-minority', 'below_p10_income', 'high_hispanic', 'high_black', 'high_asian', 'high_white']

In [23]:
def red_line_map(df,shading_var:str,redline_var:str,optional_title= None,option_zoom_params=None):
    fig, ax = plt.subplots(1, 1, figsize = (20, 10))

    # Plot data
    # Source: https://geopandas.readthedocs.io/en/latest/docs/user_guide/mapping.html
    df.plot(column = shading_var,
                           ax = ax,
                           cmap = "Greens",
                           legend = True)

    # add the the majority minority tract lines in red
    query = f'`{redline_var}`'
    df.query(query).boundary.plot(color='red',ax=ax, linewidth=2)

    # add the other tract lines in grey
    query = f'`{redline_var}` == False'    
    df.query(query).boundary.plot(color='gainsboro',ax=ax, linewidth=.2)

    # Stylize plots
    plt.style.use('bmh')

    # Set title
    if optional_title:
        plt.title(f'{shading_var} in AZ \n Red outlines are based on {redline_var}', fontdict = {'fontsize': 15})
    # optionally, zoom in
    if option_zoom_params:
        ax.set_xlim(option_zoom_params['x'])
        ax.set_ylim(option_zoom_params['y'])
        
    return fig

In [54]:
for bank in ['All Other Banks','Bank of West']:

    this_bank = bank_tract.query('which_bank == @bank')
    
    for city in citieszoom[:1]: 
        # print(city['x'], city['y'], city['city'])
        for shade in listofshading[:3]:
            for redline in listofredlining[:1]:
                
                # where to put this beaut
                pic_fname = f'''../images/{city['city']}/{bank}/SHADE {shade}/{redline}.png'''
                
                # make sure it has a home
                os.makedirs(os.path.dirname(pic_fname),
                                            exist_ok=True)

                
                # fig = red_line_map(competitor_banks,
                #                    shade, redline, "Competitor Banks: {city['city'} {shade} {redline}", city)
                # clear plot
                # f () ---> filename = f'plots/{city[0:3]}-{shade[0:10]}-{redline[0:10]}.csv
                # save plot --> plots/

../images/phx/All Other Banks/SHADE denial_rate/majority-minority.csv
../images/phx/All Other Banks/SHADE mean_approved_rate_spread/majority-minority.csv
../images/phx/All Other Banks/SHADE log(numloans)/majority-minority.csv
../images/phx/Bank of West/SHADE denial_rate/majority-minority.csv
../images/phx/Bank of West/SHADE mean_approved_rate_spread/majority-minority.csv
../images/phx/Bank of West/SHADE log(numloans)/majority-minority.csv


In [64]:
this_bank

,NAME_x,Tot.Pop,Tot.WhitePop,Tot.BlackPop,Tot.AmericIndianPop,Tot.AsianPop,Tot.NativeHawaiianPacificPop,Tot.OtherRaceAlonePop,Tot.TwoOrMoreRace,Tot.TwoOrMoreRace(Some other race),...,_merge,white_rate,majority-minority,below_p10_income,high_hispanic,high_black,high_asian,high_white,log(numloans),log(dol_loans)
1,"Census Tract 4001, Alameda County, California",3120.0,2394.0,107.0,0.0,398.0,0.0,34.0,187.0,0.0,...,both,76.730769,False,False,False,False,False,True,1.609438,13.591116
3,"Census Tract 4002, Alameda County, California",2007.0,1577.0,52.0,4.0,171.0,0.0,17.0,186.0,14.0,...,both,78.574988,False,False,False,False,False,True,0.000000,13.132314
5,"Census Tract 4003, Alameda County, California",5051.0,3748.0,459.0,0.0,590.0,23.0,134.0,97.0,0.0,...,both,74.203128,False,False,False,False,False,True,1.098612,12.983101
7,"Census Tract 4004, Alameda County, California",4007.0,2748.0,291.0,44.0,420.0,0.0,199.0,305.0,13.0,...,both,68.579985,False,False,False,False,False,True,0.000000,13.560618
9,"Census Tract 4005, Alameda County, California",4124.0,2036.0,882.0,0.0,446.0,22.0,110.0,628.0,39.0,...,both,49.369544,True,False,False,False,False,False,1.098612,11.837422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13775,"Census Tract 18.02, Yavapai County, Arizona",4754.0,4517.0,0.0,16.0,66.0,0.0,91.0,64.0,0.0,...,left_only,95.014724,False,False,False,False,False,True,0.000000,11.652687
13804,"Census Tract 11, Yuma County, Arizona",5722.0,3846.0,22.0,63.0,0.0,0.0,1671.0,120.0,49.0,...,both,67.214261,False,True,True,False,False,True,0.000000,12.180755
13816,"Census Tract 111.07, Yuma County, Arizona",2446.0,1839.0,7.0,35.0,79.0,10.0,368.0,108.0,2.0,...,both,75.183974,False,False,False,False,False,True,0.000000,11.561716
13821,"Census Tract 111.12, Yuma County, Arizona",2779.0,2504.0,0.0,38.0,40.0,16.0,102.0,79.0,0.0,...,both,90.104354,False,False,False,False,False,True,0.000000,10.126631


In [39]:
for c in bank_tract.columns:
    if 'NAME' in c:
        print(c)

NAME_x
NAME_y
NAMELSAD
